In [1]:
import pandas as pd
from dash import dash_table
import dash
import plotly
from plotly import graph_objects as go
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

In [2]:
%%time
data = pd.read_csv('../data/owid-covid-data.csv')

Wall time: 551 ms


In [3]:
# convert date to datetime dtype
data.date = pd.to_datetime(data.date)

In [4]:
# filter only needed cols
data = data[['iso_code', 'location', 'date', 'total_cases', 'new_cases', 'total_cases_per_million',
             'new_cases_per_million', 'total_deaths', 'new_deaths', 'total_deaths_per_million',
             'new_deaths_per_million', 'total_tests', 'new_tests', 'total_tests_per_thousand',
             'new_tests_per_thousand', 'total_vaccinations', 'new_vaccinations', 'total_vaccinations_per_hundred',
             'new_vaccinations_smoothed_per_million']]

In [ ]:
data.head()

In [5]:
interval = 'new'
metric = 'cases'
relative_option = ''
date_range = ['2020-03-22', '2020-04-24']

In [6]:
# filter to col and dates
table_data = data[['location', 'date', f'{interval}_{metric}']]
# convert date_range to datetimes
date_range = [pd.to_datetime(i) for i in date_range]

# create new location and date dataframe with two dates in date_range
dfs = []
for country in table_data.location.unique():
    dfs.append(pd.DataFrame(data={'location': country, 'date': date_range}))
dfs = pd.concat(dfs)

# left join data
table_data = dfs.merge(table_data, how='left', on=['location', 'date'])

# fill missing values with 0
table_data = table_data.fillna(0)

# unstack df
table_data = table_data.set_index(['location', 'date']).unstack('date')
table_data.columns = table_data.columns.droplevel()

# add absolute change col
table_data['Absolute Change'] = table_data[date_range[1]] - table_data[date_range[0]]
# def format_abs_change(x):
#     return '+' + str(x) if x >= 0 else str(x)
# table_data['Absolute Change'] = table_data['Absolute Change'].apply(format_abs_change)

# fix column names
table_data.columns = [str(table_data.columns[0])[:10], str(table_data.columns[1])[:10], table_data.columns[2]]

# add relative change col
def relative_calc(x):
    # if both 0, 0: 0
    if (x[0]==0) & (x[1] == 0):
        return 0
    # if both #, #: #
    elif (x[0]!=0) & (x[1]!=0):
        return round((x[2]/x[0])*100)
    # if 0, #: blank
    elif (x[0]==0) & (x[1]!=0):
        return 
    # if #, 0: -%100%
        return -100
    
table_data['Relative Change'] = table_data.apply(relative_calc, axis=1)

table_data

,2020-03-22,2020-04-24,Absolute Change,Relative Change
location,,,,
Afghanistan,10.0,50.0,40.0,400.0
Africa,265.0,1703.0,1438.0,543.0
Albania,13.0,15.0,2.0,15.0
Algeria,62.0,120.0,58.0,94.0
Andorra,25.0,8.0,-17.0,-68.0
...,...,...,...,...
Wallis and Futuna,0.0,0.0,0.0,0.0
World,35142.0,83928.0,48786.0,139.0
Yemen,0.0,0.0,0.0,0.0


In [ ]:
[{'name': i, 'id': i} for i in table_data.columns]

In [ ]:
table_data.to_json(orient='records')

In [ ]:
weekly_data = data.groupby(['iso_code', 'location']).rolling(7, on='date').sum().reset_index()
weekly_data

In [ ]:
app = JupyterDash(__name__)


# relative logic
if (relative_option == ['relative']) & (metric != 'vaccinations'):
    if metric == 'tests':
        col_name = f'{interval}_{metric}_per_thousand'
    else:
        col_name = f'{interval}_{metric}_per_million'
else:
    col_name = f'{interval}_{metric}'

# filter to col and dates
table_data = data[['location', 'date', col_name]]
# convert date_range to datetimes
date_range = [pd.to_datetime(i) for i in date_range]

# create new location and date dataframe with two dates in date_range
dfs = []
for country in table_data.location.unique():
    dfs.append(pd.DataFrame(data={'location': country, 'date': date_range}))
dfs = pd.concat(dfs)

# left join data
table_data = dfs.merge(table_data, how='left', on=['location', 'date'])

# fill missing values with 0
table_data = table_data.fillna(0)

# unstack df
table_data = table_data.set_index(['location', 'date']).unstack('date')
table_data.columns = table_data.columns.droplevel()

# add absolute change col
table_data['Absolute Change'] = table_data[date_range[1]] - table_data[date_range[0]]

# fix column names
table_data.columns = [str(table_data.columns[0])[:10], str(table_data.columns[1])[:10], table_data.columns[2]]

# add relative change col
def relative_calc(x):
    # if both 0, 0: 0
    if (x[0] == 0) & (x[1] == 0):
        return 0
    # if both #, #: #
    elif (x[0] != 0) & (x[1] != 0):
        return round((x[2] / x[0]) * 100)
    # if 0, #: blank
    elif (x[0] == 0) & (x[1] != 0):
        return
        # if #, 0: -%100%
        return -100

table_data['Relative Change'] = table_data.apply(relative_calc, axis=1)


app.layout = html.Div(
    dash_table.DataTable(
        id='data-table',
        columns = [{'name': i, 'id': i} for i in table_data.columns],
        data = table_data.to_dict('records')
    )
)


app.run_server(mode='inline', debug=True)

In [ ]:
help(dash_table)